# Sentiment Analysis for Financial News

In [378]:
import pandas as pd
import numpy as np

# cleaning
import unidecode

# stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# vocab
from collections import Counter

# models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from tqdm import tqdm  


[nltk_data] Downloading package stopwords to C:\Users\Camille
[nltk_data]     Leempoels\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocessing

Loading the dataset.

In [343]:
data = pd.read_csv('data.csv', encoding='latin-1', names=['sentiment','text'])

In [344]:
data.head()

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


Cleaning the text.

In [345]:
# Removing punctuation and special caracters
data['text'] = data['text'].str.replace('[^\w\s]', '', regex=True)
data['text'] = data['text'].str.replace('_', ' ')
data['text'] = data['text'].astype('unicode')
data['text'] = data['text'].transform(lambda x: unidecode.unidecode(x))

# Lowercase the text
data['text'] = data['text'].str.lower()

# Removing numbers
data['text'] = data['text'].str.replace('\d+', '', regex=True)

# Removing stop words
nltk_stopwords = stopwords.words('english')
data['text'] = data['text'].apply(lambda x: [item for item in x.split() if item not in nltk_stopwords])

# Lemmatization and/or Expand Contractions ?

Split train and test set

In [346]:
_train_set_, _test_set_ = train_test_split(data, test_size=0.2, random_state=1)

Building the vocabulary from the training set

In [347]:
# extract all the words and count them
vocab = Counter([word for sentence in _train_set_['text'].values.tolist() for word in sentence])

print('Number of words in the training set : ' + str(sum(vocab.values())))
print('Number of unique words in the training set : ' + str(len(list(vocab))))

# Sort the words
vocab = vocab.most_common()

print('Most frequent words : ' + str(vocab[0:10]))

# convert the vocabulary to a Python Dict
vocab = dict(vocab)

# encode words as integers
sparse_vocab = {word:i for i, word in enumerate(vocab, 1)} 

Number of words in the training set : 46400
Number of unique words in the training set : 8337
Most frequent words : [('eur', 1057), ('company', 680), ('mn', 475), ('said', 437), ('finnish', 405), ('sales', 354), ('million', 335), ('profit', 325), ('net', 323), ('finland', 278)]


Encode the text and labels

In [348]:
def encode_text(data):
    # encode sentences and set -1 if the word is not in the vocabulary
    data['text_encoded'] = data['text'].apply(
        lambda sentence : [sparse_vocab.get(word, -1) for word in sentence]
    )

    # drop sentences that are extremely short or long
    data['length'] = data['text_encoded'].apply(lambda sentence : len(sentence))
    limit_low, limit_high  = data['length'].quantile(q=[0.01, 0.99]).values
    data = data[(data['length'] <= limit_high) & (data['length'] >= limit_low)].copy()

    # pad sentences so that each vector has the same length
    max_len = int(limit_high)
    data['text_encoded'] = data.apply(lambda row: row['text_encoded'] + [0] * (max_len - row['length']), axis=1)
    data['length'] = data['text_encoded'].apply(lambda sentence : len(sentence))

    if sum(data['length'] != max_len):
        print('WARNING')

    return data

train_set_txt = encode_text(_train_set_.copy())
test_set_txt = encode_text(_test_set_.copy())

# train_set_labels = (
#     OneHotEncoder(
#         handle_unknown='ignore', 
#         sparse_output=False
#     )
#     .fit_transform(
#         train_set_txt['sentiment']
#         .values
#         .reshape(-1, 1)
#     )
# )

# test_set_labels = (
#     OneHotEncoder(
#         handle_unknown='ignore', 
#         sparse_output=False
#     )
#     .fit_transform(
#         test_set_txt['sentiment']
#         .values
#         .reshape(-1, 1)
#     )
# )

train_set_labels = train_set_txt['sentiment'].replace(['positive','neutral','negative'], [0, 1, 2])
test_set_labels = test_set_txt['sentiment'].replace(['positive','neutral','negative'], [0, 1, 2])

## Supervised learning

In [349]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [351]:
class RNN_model(nn.Module):

    def __init__(self, vocab_dim, embedding_dim, hidden_dim):
        super(RNN_model, self).__init__()

        # embedding layer
        self.embedding = nn.Embedding(vocab_dim, embedding_dim)
        # RNN layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        # fully connected layer
        self.fc = nn.Linear(hidden_dim, 3)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, txt):
        emb_txt = self.embedding(txt.T)
        rnn_out, rnn_hidden = self.rnn(emb_txt)
        fc_out = self.fc(rnn_hidden.squeeze(0))
        output = self.softmax(fc_out)

        return fc_out

In [390]:
def train_loop(dataloader, model, optimizer, loss_function, accuracy_function):
    
    loss_hist = np.array([])
    accuracy_hist = np.array([])
    
    model.train()
    
    for txt, label in dataloader:

        txt, label = txt.to(device), label.to(device)

        # forward computation
        pred = model(txt)
        loss = loss_function(pred, label.squeeze())
        accuracy = accuracy_function(pred, label)

        # backward computation
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()

        # metrics
        loss_hist = np.append(loss_hist, loss.item())
        accuracy_hist = np.append(accuracy_hist, accuracy.item())
        
    return np.mean(loss_hist), np.mean(accuracy_hist)

In [377]:
def eval_model(dataloader, model, loss_function, accuracy_function):
    
    loss_hist = np.array([])
    accuracy_hist = np.array([])
    model.eval()
    
    with torch.no_grad():
    
        for txt, label in dataloader:

            txt, label = txt.to(device), label.to(device)

            # forward 
            pred = model(txt)
            loss = loss_function(pred, label)
            accuracy = accuracy_function(pred, label)

            # metrics
            loss_hist = np.append(loss_hist, loss.item())
            accuracy_hist = np.append(accuracy_hist, accuracy.item())
        
    return np.mean(loss_hist), np.mean(accuracy_hist)

In [393]:
VOCAB_DIM = len(vocab) + 1
EMBEDDING_DIM = 100
HIDDEN_DIM = 32
LEARNING_RATE = 1e-2
BATCH_SIZE = 32
EPOCHS = 100

train_set = TensorDataset(
    torch.tensor(np.vstack(train_set_txt['text_encoded'])), 
    torch.tensor(np.vstack(train_set_labels)).long()
)

test_set = TensorDataset(
    torch.tensor(np.vstack(test_set_txt['text_encoded'])), 
    torch.tensor(np.vstack(test_set_labels)).long()
)

train_dataloader = DataLoader(
    train_set, 
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dataloader = DataLoader(
    test_set, 
    batch_size = BATCH_SIZE,
    shuffle = True
)

model = RNN_model(VOCAB_DIM, EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.CrossEntropyLoss().to(device)
accuracy_function = lambda pred, target : torch.sum((torch.argmax(torch.softmax(pred, dim=1), dim=1) == target.T)) / len(target)


In [394]:
for epoch in tqdm(range(EPOCHS)):
    train_loss, train_accuracy = train_loop(train_dataloader, model, optimizer, loss_function, accuracy_function)
    # test_loss, test_accuracy = eval_model(test_dataloader, model, loss_function, accuracy_function)
    print('EPOCH : ' + str(epoch + 1))
    print('Training loss : ' + str(train_loss) + ' | Training accuracy : ' + str(train_accuracy))
    # print('Testing loss : ' + str(test_loss) + ' | Testing accuracy : ' + str(test_accuracy))

  1%|          | 1/100 [00:00<00:40,  2.42it/s]

EPOCH : 1
Training loss : 0.9323620806137721 | Training accuracy : 0.591493055721124


  2%|▏         | 2/100 [00:00<00:41,  2.35it/s]

EPOCH : 2
Training loss : 0.9315522039930025 | Training accuracy : 0.5900173611938954


  3%|▎         | 3/100 [00:01<00:41,  2.32it/s]

...

EPOCH : 98
Training loss : 0.9245314598083496 | Training accuracy : 0.5980902781089147


 99%|█████████▉| 99/100 [00:42<00:00,  2.39it/s]

EPOCH : 99
Training loss : 0.9269091968735059 | Training accuracy : 0.5955729166666667


100%|██████████| 100/100 [00:43<00:00,  2.31it/s]

EPOCH : 100
Training loss : 0.9242279320955277 | Training accuracy : 0.5989583333333334
